In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
import findspark
findspark.init()

In [3]:
clientdata = pd.read_csv("./cliente_tabla.csv/cliente_tabla.csv")
productdata = pd.read_csv("./producto_tabla.csv/producto_tabla.csv")
townstate = pd.read_csv("./town_state.csv/town_state.csv")


In [ ]:
traindata = pd.read_csv("./train.csv/train.csv")

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [5]:
traindata = spark.read.csv("./train.csv/train.csv",header=True,inferSchema=True)

In [8]:
traindata.printSchema()

root
 |-- Semana: integer (nullable = true)
 |-- Agencia_ID: integer (nullable = true)
 |-- Canal_ID: integer (nullable = true)
 |-- Ruta_SAK: integer (nullable = true)
 |-- Cliente_ID: integer (nullable = true)
 |-- Producto_ID: integer (nullable = true)
 |-- Venta_uni_hoy: integer (nullable = true)
 |-- Venta_hoy: double (nullable = true)
 |-- Dev_uni_proxima: integer (nullable = true)
 |-- Dev_proxima: double (nullable = true)
 |-- Demanda_uni_equil: integer (nullable = true)



In [29]:
from pyspark.sql.functions import max,count
#traindata.select(max('Semana')).show()

In [16]:
traindata.select(max('Agencia_ID'), max('Canal_ID'), max('Ruta_SAK'), max('Cliente_ID'), max('Producto_ID'), max('Venta_uni_hoy'),
                max('Dev_uni_proxima')).show()

+---------------+-------------+-------------+---------------+----------------+------------------+--------------------+
|max(Agencia_ID)|max(Canal_ID)|max(Ruta_SAK)|max(Cliente_ID)|max(Producto_ID)|max(Venta_uni_hoy)|max(Dev_uni_proxima)|
+---------------+-------------+-------------+---------------+----------------+------------------+--------------------+
|          25759|           11|         9991|     2015152015|           49997|              7200|              250000|
+---------------+-------------+-------------+---------------+----------------+------------------+--------------------+



In [5]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType, DoubleType, ShortType, ByteType
data_schema = [StructField("WeekNumber", ByteType(), True),StructField("DepotID", ShortType(), True),
              StructField("SalesChannelID", ByteType(), True),StructField("RouteID", ShortType(), True),
              StructField("ClientID", IntegerType(), True),StructField("ProductID", ShortType(), True),
              StructField("SalesUnits", ShortType(), True),StructField("Sales", DoubleType(), True),
              StructField("ReturnUnits", IntegerType(), True),StructField("Returns", DoubleType(), True),
              StructField("Demand", IntegerType(), True)]
final_struc = StructType(fields=data_schema)
df = spark.read.csv("./train.csv/train.csv", schema=final_struc)

In [6]:
df.printSchema()

root
 |-- WeekNumber: byte (nullable = true)
 |-- DepotID: short (nullable = true)
 |-- SalesChannelID: byte (nullable = true)
 |-- RouteID: short (nullable = true)
 |-- ClientID: integer (nullable = true)
 |-- ProductID: short (nullable = true)
 |-- SalesUnits: short (nullable = true)
 |-- Sales: double (nullable = true)
 |-- ReturnUnits: integer (nullable = true)
 |-- Returns: double (nullable = true)
 |-- Demand: integer (nullable = true)



In [32]:
df.groupBy("SalesChannelID").count().show()

+--------------+--------+
|SalesChannelID|   count|
+--------------+--------+
|          null|       1|
|             1|67435217|
|             6|  281389|
|             5|  145818|
|             9|     378|
|             4| 3757773|
|             8|   66970|
|             7|  671128|
|            11|  982295|
|             2|  839496|
+--------------+--------+



In [8]:
train = df.filter(df['SalesChannelID'] == 11).toPandas()

In [9]:
train.to_csv("traindata.csv")

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982295 entries, 0 to 982294
Data columns (total 11 columns):
WeekNumber        982295 non-null int8
DepotID           982295 non-null int16
SalesChannelID    982295 non-null int8
RouteID           982295 non-null int16
ClientID          982295 non-null int32
ProductID         528498 non-null float64
SalesUnits        982295 non-null int16
Sales             982295 non-null float64
ReturnUnits       982295 non-null int32
Returns           982295 non-null float64
Demand            982295 non-null int32
dtypes: float64(3), int16(3), int32(3), int8(2)
memory usage: 41.2 MB


In [11]:
clientdata.head()

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [22]:
clientdata[clientdata['NombreCliente'].apply(lambda x : 'OXXO' in  x)]

,Cliente_ID,NombreCliente
1,1,OXXO XINANTECATL
47,46,OXXO SAN CARLOS
1228,5408,OXXO LA MORA
2377,8487,OXXO 5 DE MAYO
2509,8825,OXXO SAN SEBASTIAN
...,...,...
935353,10351796,OXXO ALAMO
935354,10351802,OXXO IDOLO II
935355,10351808,OXXO TUXPAM TAMIAHUA
935356,10351814,OXXO DE LA LLAVE


In [19]:
productdata.head()

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


In [21]:
len(productdata[productdata['NombreProducto'].apply(lambda x : 'Div Tira Mini Doradita' in  x)])

8

There are multiple repititions of same/similar client name and similar product names

In [41]:
clientid = clientdata.groupby('Cliente_ID').count()
dupliClientID = clientid[clientid['NombreCliente'] > 1].index.values

In [44]:
duplicatedata = clientdata[clientdata['Cliente_ID'].apply(lambda x : x in dupliClientID)]

In [48]:
x = 0
for i in range(0,9000, 2):
    if duplicatedata.iloc[i]['NombreCliente'] != duplicatedata.iloc[i+1]['NombreCliente']:
       x = i 

In [49]:
x

8998

For duplicate client ids, the client names are same, so we can delete the duplicate data safely